In [40]:
import requests
import json
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, merge, concat
from dotenv import load_dotenv
from altair import Chart, Color, Y, X, Tooltip, Column, Axis

In [75]:
filepath = '/Users/jasongersing/PycharmProjects/fastApiProject/.env'
load_dotenv(filepath)
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [76]:
db_GB = MongoClient(url)[database][collection_GB]
db_GS = MongoClient(url)[database][collection_GS]

In [77]:
grape_buyers = DataFrame(db_GB.find(projection={"_id": False, "grapes_seeking": True, "volume_seeking": True, "state": True}))
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])
grape_buyers.rename(columns={"grapes_seeking": "variety", "volume_seeking": "tons"}, inplace=True)
grape_buyers["side"] = "Buyer"
grape_buyers["tons"] = Series([int(num) for num in grape_buyers["tons"]])
grape_buyers = grape_buyers.groupby(["variety", "side", "state"]).agg({"tons": "sum"}).reset_index()
grape_buyers.head()

,variety,side,state,tons
0,Cabernet Sauvignon,Buyer,California,125
1,Cabernet Sauvignon,Buyer,Oregon,70
2,Cabernet Sauvignon,Buyer,Washington,100
3,Chardonnay,Buyer,California,125
4,Chardonnay,Buyer,Oregon,255


In [78]:
grape_sellers = DataFrame(db_GS.find(projection={"_id": False, "grapes_selling": True, "volume_selling": True, "state": True}))
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])
grape_sellers.rename(columns={"grapes_selling": "variety", "volume_selling": "tons"}, inplace=True)
grape_sellers["side"] = "Seller"
grape_sellers["tons"] = Series([int(num) for num in grape_sellers["tons"]])
grape_sellers = grape_sellers.groupby(["variety", "side", "state"]).agg({"tons": "sum"}).reset_index()
grape_sellers.head()

,variety,side,state,tons
0,Cabernet Sauvignon,Seller,California,10
1,Cabernet Sauvignon,Seller,Oregon,175
2,Cabernet Sauvignon,Seller,Washington,40
3,Chardonnay,Seller,California,25
4,Chardonnay,Seller,Oregon,175


In [79]:
chart_buyer = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-state-buyer').json()
Chart.from_dict(chart_buyer)

alt.Chart(...)

In [80]:
chart_seller = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-state-seller').json()
Chart.from_dict(chart_seller)

alt.Chart(...)

In [81]:
total = concat([grape_buyers, grape_sellers])
total.head()

,variety,side,state,tons
0,Cabernet Sauvignon,Buyer,California,125
1,Cabernet Sauvignon,Buyer,Oregon,70
2,Cabernet Sauvignon,Buyer,Washington,100
3,Chardonnay,Buyer,California,125
4,Chardonnay,Buyer,Oregon,255


In [84]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")

In [85]:
def grouped_bar_chart(df: DataFrame, column_1: str, column_2: str, column_3: str, column_4: str) -> Chart:
    return Chart(
        df,
        title=f"{title_fix(column_1)} per {title_fix(column_3)} by {title_fix(column_2)} by {title_fix(column_4)}",
    ).mark_bar().encode(
        column=Column(column_4),
        x=X(column_3, title=title_fix(column_3), sort="-y"),
        y=Y(column_1),
        color=Color(column_2, title=title_fix(column_2)),
        tooltip=Tooltip([column_2, column_3, column_1])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )


In [86]:
grouped_bar_chart(total, 'tons', 'side', 'variety', 'state')

alt.Chart(...)

In [87]:
chart = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-state-combined').json()
Chart.from_dict(chart)

alt.Chart(...)